In [ ]:
import librosa
import os
import numpy as np

In [ ]:
Class Augmenter():
    
    def __init__(self,numpy_array,sr):
        self.numpy_array = numpy_array
        self.sr = sr
        
    def padd(self,interval):
        return numpy_array,sr
    
    def change_pitch(self,magnitude):
        # magnitude = x. 0<x<1 slows down, x=1 identity, x>1 speeds up
        return numpy_array,sr
    
    def change_speed(self,magnitude):
        # magnitude = x. 0<x<1 slows down, x=1 identity, x>1 speeds up
        return numpy_array,sr
    
    def change_volume(self,magnitude):
        # magnitude = x. 0<x<1 quieter, x=1 identity, x>1 louder
        return numpy_array,sr
    

        

In [ ]:
def main():
    pth = ""
    #play audio
    arr,sr = librosa.load(pth)
    aug = Augmenter(arr,sr)
    aug_2,sr_2 = aug.padd(100)
    #play audio
    
    
if __name__ == "__main__":
    main()